In [1]:
import os
import sys
from sys import platform
import pandas as pd
import datetime
BASE_PATH = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(BASE_PATH+'\src' if platform == 'win32' else BASE_PATH+'/src')
from config.global_config import get_global_config
from utils import get_total_iterations

In [2]:
_config = get_global_config()
MODELS = sorted(_config.get('MODELS'))
HP_CONFIG_PATH = os.path.join(BASE_PATH, _config.get('HP_CONFIG_PATH'))
RESULTS_PATH = os.path.join(BASE_PATH, _config.get('RESULT_PATH'))
RESULTS_PATH_SHARED = os.path.join(BASE_PATH, _config.get('RESULT_PATH_SHARED'))
if platform == 'win32':
	HP_CONFIG_PATH = HP_CONFIG_PATH.replace('/', '\\')
	RESULTS_PATH = RESULTS_PATH.replace('/', '\\')
	RESULTS_PATH_SHARED = RESULTS_PATH_SHARED.replace('/', '\\')
EMISSIONS_FILE = 'emissions.csv'
METRICS_FILE = 'metrics.csv'
PARAMS_FILE = 'params.csv'

In [3]:
print('\n' + ''.join(['> ' for i in range(33)]))
print(f'\n{"MODEL":<15}{"N_RUN":>10}{"":>10}{"MODEL":<15}{"N_RUN":>10}\n')
print(''.join(['> ' for i in range(33)]))
for n in range(0, len(MODELS), 2):
	n1 = get_total_iterations(HP_CONFIG_PATH + MODELS[n] + ".hyper")
	if n + 1 < len(MODELS):
		n2 = get_total_iterations(HP_CONFIG_PATH + MODELS[n + 1] + ".hyper")
		print(f'{MODELS[n]:<15}{n1:>10}{"":>10}{MODELS[n + 1]:<15}{n2:>10}')
	else:
		print(f'{MODELS[n]:<15}{n1:>10}')


> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

MODEL               N_RUN          MODEL               N_RUN

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
ADMMSLIM            20736          BPR                    27
CDAE                 1296          CFKG                  162
CKE                   243          ConvNCF             20736
DGCF                  243          DMF                    81
DiffRec           2985984          EASE                   54
ENMF                 2592          FISM                   54
GCMC                 1944          ItemKNN               216
KGAT                18225          KGCN                  729
KGIN                 1296          KGNNLS                162
KTUP                23328          LDiffRec        191102976
LINE                  162          LightGCN              216
MCCLK                6480          MKR                  5184
MacridVAE            5184          MultiDAE              162
MultiVAE 

In [4]:
experiments = []
for d in next(os.walk(RESULTS_PATH))[1]:
	for m in next(os.walk(RESULTS_PATH + d))[1]:
		experiments.append(os.path.join(d, m))
experiments = sorted(experiments)
results = {
	'emissions': [],
	'duration': []
}

In [5]:
print(''.join(['> ' for i in range(42)]))
print(f'\n{"ID":<5}{"DATASET":<18}{"MODEL":<8}{"RUNs":>8}{"DURATION":>20}{"AVG DURATION":>18}\n')
print(''.join(['> ' for i in range(42)]))
for i, v in enumerate(experiments):
	emissions_df = pd.read_csv(os.path.join(RESULTS_PATH, v, EMISSIONS_FILE))
	e = emissions_df[['project_name', 'duration', 'emissions']].groupby(['project_name']).sum()
	if len(e) > 1 or len(m) > 1:
		raise Exception()
	else:
		results['emissions'].append(e.iloc[0]['emissions']*1000)
		results['duration'].append(e.iloc[0]['duration'])
		runs = get_total_iterations(HP_CONFIG_PATH + v.split("/")[1] + ".hyper")
		print(f'\n{i+1:<5}{v.split("/")[0]+"_100k":<18}{v.split("/")[1]:<8}{runs:>8}{str(datetime.timedelta(seconds=int(e.iloc[0]["duration"]))):>20}{str(datetime.timedelta(seconds=int(e.iloc[0]["duration"]/runs)))[2:]:>18}')

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

ID   DATASET           MODEL       RUNs            DURATION      AVG DURATION

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 


Exception: 

In [7]:
experiments = []
for d in next(os.walk(RESULTS_PATH_SHARED))[1]:
	for m in next(os.walk(RESULTS_PATH_SHARED + d))[1]:
		experiments.append(os.path.join(d, m))
experiments = sorted(experiments)
results = {
	'emissions': [],
	'duration': []
}

In [9]:
print(''.join(['> ' for i in range(40)]))
print(f'\n{"ID":<5}{"DATASET":<18}{"MODEL":<12}{"DURATION(h)":>15}{"EMISSIONS(g)":>18}\n')
print(''.join(['> ' for i in range(40)]))
for i, v in enumerate(experiments):
	e = pd.read_csv(os.path.join(RESULTS_PATH_SHARED, v, EMISSIONS_FILE))
	results['emissions'].append(e.iloc[0]['emissions']*1000)
	results['duration'].append(e.iloc[0]['duration'])
	print(f'{i+1:<5}{v.split("/")[0][:12]:<18}{v.split("/")[1]:<12}{str(datetime.timedelta(seconds=int(e.iloc[0]["duration"]))):>15}{e.iloc[0]["emissions"]*1000:>18.5f}')

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 

ID   DATASET           MODEL           DURATION(h)      EMISSIONS(g)

> > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > > 
1    amazon_books      ItemKNN             0:03:03           0.09906
2    mind              BPR                 0:03:22           0.07977
3    mind              CDAE                0:03:26           0.08848
4    mind              DGCF                3:31:22           9.20494
5    mind              DMF                 0:19:49           0.76172
6    mind              DiffRec             0:07:15           0.14438
7    mind              EASE                0:00:48           0.01958
8    mind              ENMF                0:04:53           0.13767
9    mind              FISM                1:10:48           2.09903
10   mind              GCMC               10:16:05          17.71566
11   mind              ItemKNN             0:01:22           0.03872
12   min